<a href="https://colab.research.google.com/github/sahil3Vedi/Enron-Email-Classifier/blob/master/Response_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os
import numpy as np
from sklearn.svm import LinearSVC
import random
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.utils import to_categorical

Using TensorFlow backend.


In [0]:
def preprocess(x, y):
  x = tf.cast(x, tf.float32) / 255.0
  y = tf.cast(y, tf.int64)

  return x, y

def create_dataset(xs, ys, n_classes=10):
  ys = tf.one_hot(ys, depth=n_classes)
  return tf.data.Dataset.from_tensor_slices((xs, ys)) \
    .map(preprocess) \
    .shuffle(len(ys)) \
    .batch(500)

In [0]:
#TRAINING

ENRONFOLDER = 'drive/My Drive/Enron Spam/'
ENRONNOSPAMDATASET = 'drive/My Drive/Enron Spam/ham'

ANNO_LOC = 'drive/My Drive/Enron Spam/annotations.npy'
ANNOIND_LOC = 'drive/My Drive/Enron Spam/annotation_indices.npy'
CONTEXT_LOC = 'drive/My Drive/Enron Spam/context_annotations.npy'
CONTEXTIND_LOC = 'drive/My Drive/Enron Spam/context_annotation_indices.npy'
FEATURES_LOC = 'drive/My Drive/Enron Spam/featureVectors.npy'

print("All vectors loaded!")

All vectors loaded!


In [0]:
feature_vectors = np.load(FEATURES_LOC)
formal_informal_annotations = np.load(ANNO_LOC)
formal_informal_indices = np.load(ANNOIND_LOC)
context_annotations = np.load(CONTEXT_LOC)
context_annotations_indices = np.load(CONTEXTIND_LOC)

In [0]:
formal_informal_X = []
for each_index in formal_informal_indices:
  formal_informal_X.append(feature_vectors[each_index])
X_TRAIN_1 = np.asarray(formal_informal_X)
Y_LOAD = [each_element for each_element in formal_informal_annotations]
Y_TRAIN_1 = to_categorical(Y_LOAD)

In [0]:
context_X = []
for each_index in context_annotations_indices:
  context_X.append(feature_vectors[each_index])
X_TRAIN_2 = np.asarray(context_X)
Y_TRAIN_2 = to_categorical(context_annotations)

In [0]:
print(np.shape(X_TRAIN_1))
print(np.shape(Y_TRAIN_1))
print(np.shape(X_TRAIN_2))
print(np.shape(Y_TRAIN_2))

(50, 1000)
(50, 2)
(50, 1000)
(50, 4)


In [0]:
#MODEL 1: Formal-Informal_Classifier

model1 = tf.keras.Sequential([
# Adds a densely-connected layer with 64 units to the model:
layers.Dense(64, activation='relu', input_shape=(1000,)),
# Add another:
layers.Dense(64, activation='relu'),
# Add an output layer with 2 output units:
layers.Dense(2, activation='softmax')])

model1.compile(optimizer=tf.keras.optimizers.Adam(0.01),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model1.fit(X_TRAIN_1, Y_TRAIN_1, epochs=10, batch_size=32)

Epoch 1/10
2/2 [==============================] - 0s 4ms/step - loss: 0.7047 - accuracy: 0.4800
Epoch 2/10
2/2 [==============================] - 0s 3ms/step - loss: 0.5877 - accuracy: 0.7200
Epoch 3/10
2/2 [==============================] - 0s 3ms/step - loss: 0.4923 - accuracy: 0.9000
Epoch 4/10
2/2 [==============================] - 0s 2ms/step - loss: 0.4087 - accuracy: 0.9800
Epoch 5/10
2/2 [==============================] - 0s 2ms/step - loss: 0.3552 - accuracy: 1.0000
Epoch 6/10
2/2 [==============================] - 0s 2ms/step - loss: 0.3249 - accuracy: 1.0000
Epoch 7/10
2/2 [==============================] - 0s 2ms/step - loss: 0.3163 - accuracy: 1.0000
Epoch 8/10
2/2 [==============================] - 0s 3ms/step - loss: 0.3139 - accuracy: 1.0000
Epoch 9/10
2/2 [==============================] - 0s 2ms/step - loss: 0.3135 - accuracy: 1.0000
Epoch 10/10
2/2 [==============================] - 0s 2ms/step - loss: 0.3133 - accuracy: 1.0000


In [0]:
#MODEL 2: Context_Classifier

model2 = tf.keras.Sequential([
# Adds a densely-connected layer with 64 units to the model:
layers.Dense(64, activation='relu', input_shape=(1000,)),
# Add another:
layers.Dense(100, activation='relu'),
# Add an output layer with 4 output units:
layers.Dense(4, activation='softmax')])

model2.compile(optimizer=tf.keras.optimizers.Adam(0.01),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model2.fit(X_TRAIN_2, Y_TRAIN_2, epochs=10, batch_size=25)

Epoch 1/10
2/2 [==============================] - 0s 3ms/step - loss: 1.3633 - accuracy: 0.3800
Epoch 2/10
2/2 [==============================] - 0s 3ms/step - loss: 1.1440 - accuracy: 0.5400
Epoch 3/10
2/2 [==============================] - 0s 3ms/step - loss: 0.9582 - accuracy: 0.8200
Epoch 4/10
2/2 [==============================] - 0s 2ms/step - loss: 0.8716 - accuracy: 0.9200
Epoch 5/10
2/2 [==============================] - 0s 2ms/step - loss: 0.8173 - accuracy: 0.9600
Epoch 6/10
2/2 [==============================] - 0s 2ms/step - loss: 0.7893 - accuracy: 0.9800
Epoch 7/10
2/2 [==============================] - 0s 4ms/step - loss: 0.7780 - accuracy: 0.9600
Epoch 8/10
2/2 [==============================] - 0s 2ms/step - loss: 0.7499 - accuracy: 1.0000
Epoch 9/10
2/2 [==============================] - 0s 2ms/step - loss: 0.7463 - accuracy: 1.0000
Epoch 10/10
2/2 [==============================] - 0s 2ms/step - loss: 0.7489 - accuracy: 1.0000


In [0]:
#Generating Responses for random mails:

def fetchLabel(mailIndex):
  twoD_array = []
  classification1 = ""
  classification2 = ""
  responseHeader = ""
  responseFooter = ""
  R1 = []
  R2 = []
  feature_vector = feature_vectors[mailIndex]
  twoD_array.append(feature_vector)
  file_location = os.listdir(ENRONNOSPAMDATASET)[mailIndex]
  FILENAME = os.path.join(ENRONNOSPAMDATASET, file_location)
  DD = np.asarray(twoD_array)
  with open(FILENAME, 'r',encoding='utf-8',errors='ignore') as myfile:
    data = myfile.read()
    print("Message Contents: ")
    print(data)

    result1 = model1.predict(DD)
    R1 = [x for x in result1[0]]
    maxInd1 = R1.index(max(R1))
    if(maxInd1==0):
      classification1="formal"
      responseFooter = "Yours Sincerely.."
    elif(maxInd1==1):
      classification1="informal"
      responseFooter = "Regards.."
    else:
      print("Error in Formal-Informal Classification")
    
    result2 = model2.predict(DD)
    R2 = [y for y in result2[0]]
    maxInd2 = R2.index(max(R2))
    if(maxInd2==0):
      classification2="affirmative"
      responseHeader = "(a) Thank You, I will look into it (b) Thank you for informing"
    elif(maxInd2==1):
      classification2="proposal"
      responseHeader = "(a) Sure, we can proceed (b) I am not sure we can proceed"
    elif(maxInd2==2):
      classification2="followup"
      responseHeader="(a) Okay, I will follow up as required (b) Unfortunately, I am unable to follow up"
    elif(maxInd2==3):
      classification2="invite"
      responseHeader="(a) Thank You, I will be there (b) Thank you, but I won't be able to make it"
    else:
      print("Error in context classification")

    fetch_results = [classification1,classification2]

    #generating response based on category:

    print("\nResponse:")
    print(responseHeader)
    print(responseFooter)

    return fetch_results

In [0]:
genIndex = random.randrange(3000)
fetchLabel(genIndex)

Message Contents: 
Subject: re : meter 984229 - roos common point - trade zone 18
mary ,
hplc currently has a gas purchase agreement ( global # 96014987 ) in place with
calpine that extends through 5 / 31 / 2001 . sitara deal no . 133259 correctly has
this contract attached to it . effective 11 / 1 / 1998 the contract was amended
releasing gouger wells no . 5 & 6 . these wells were apparently the only
producing wells behind this point . evidence being no measured volumes in
mips . there are however other non producing wells behind this point . is
your sept
however , they were not added behind meter 6835 . therefore , i do not agree
that the deal ( 133259 ) should be edited to change the meter .
hopefully this information is useful !
vlt
x 3 - 6353
mary poorman @ enron
10 / 23 / 2000 02 : 06 pm
to : katherine benedict / hou / ect @ ect , julie meyers / hou / ect @ ect , vance l
taylor / hou / ect @ ect , daren j farmer / hou / ect @ ect , pat clynes / corp / enron @ enron
cc :
subject :

['formal', 'invite']